In [ ]:
import numpy as np
import pandas as pd
from tools.greedy import Greedy
import os
import random

# Comparamos la solución previa con el frente de pareto
 la solución actual con el frente de pareto, ponderamos la dierencia de cada f entre 0 y 1 (cuanto tendría que mejorar d1 para que fuera solución, cuanto d2 para que fuera solución y cuánto d3)

Por ejemplo, solución previa 450, 220, 17. si el 450 fuera 437, sería solución. entonces 1-(437/450). lo mismo con el resto.
La nueva, sacamos lo mismo, 400, 230 y 27. si el 400 fuera 397, sería solución. (397/400)+...

La recompensa será (valor_solucion_actual-valor_solucion_previa)/3. Si se acerca mucho a soluciones, el valor de recompensa será alto.

Si saca una solución exacta del frente de pareto, finalizar y dar un -x?

Si saca una solución nueva, recompensar y dar un +x

In [ ]:



####
# F1#------------------------------------------------------------------------------------------
####


def f1(supply_selected, df_distances_demand):
    """
    - df_distances_demand: DataFrame con las distancias entre los puntos de suministro y los puntos de demanda.
    - supply_selected: Lista con los indices de los puntos de suministro seleccionados.
    """

    value = df_distances_demand.iloc[:, supply_selected].min(axis=1).max()
    return value


def max_min_dist(supply_selected, df_distances_demand):
    """
    Calcula la máxima de las mínimas distancias de cada punto de demanda a los puntos de suministro seleccionados.
    """
    return df_distances_demand.iloc[:, supply_selected].min(axis=1).max()


def local_search_f1(solution, value, m, df_distances_demand, n_veces):
    """
    Realiza una búsqueda local para mejorar la solución utilizando el enfoque optimizado de max_min_dist.
    """
    count=1
    count_until_improve=0

    best_solution = solution[:]
    best_objective = value
    len_solution=len(solution)

    while True:
        for i in range(len_solution):
            improved=False
            for j in range(m):
                if j not in solution:
                    temp_solution = solution[:]
                    temp_solution[i] = j
                    temp_objective = max_min_dist(temp_solution, df_distances_demand)

                    if temp_objective < best_objective:
                        best_objective = temp_objective
                        best_solution = temp_solution[:]
                        improved=True
                        count+=1

            if count==n_veces:
                return best_solution, best_objective
            if improved:
                count_until_improve=0
            else:
                count_until_improve+=1
                if count_until_improve==len_solution:
                    return best_solution, best_objective



####
# F2#------------------------------------------------------------------------------------------
####


def f2(supply_selected, df_distances_demand):
    """
    - df_distances_demand: DataFrame con las distancias entre los puntos de suministro y los puntos de demanda.
    - supply_selected: Lista con los indices de los puntos de suministro seleccionados.
    """
    asignacion = df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
    maximum = asignacion.value_counts().max()
    return maximum


def max_demand_per_supply(supply_selected, df_distances_demand):
    """
    Calcula el número máximo de demandas asignadas a un único punto de suministro.
    """
    asignacion = df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
    return asignacion.value_counts().max()


def local_search_f2(solution, value, m, df_distances_demand, n_veces):
    """
    Realiza una búsqueda local para mejorar la solución minimizando el máximo número de demandas
    asignadas a un único punto de suministro.
    """
    count=1
    count_until_improve=0

    best_solution = solution[:]
    best_objective = value

    len_solution=len(solution)
    
    while True:
        for i in range(len_solution):
            improved=False
            for j in range(m):
                if j not in solution:
                    temp_solution = solution[:]
                    temp_solution[i] = j
                    temp_objective = max_demand_per_supply(
                        temp_solution, df_distances_demand
                    )

                    if temp_objective < best_objective:
                        best_objective = temp_objective
                        best_solution = temp_solution[:]
                        improved=True
                        count+=1

            if count==n_veces:
                return best_solution, best_objective
            if improved:
                count_until_improve=0
            else:
                count_until_improve+=1
                if count_until_improve==len_solution:
                    return best_solution, best_objective


####
# F3#------------------------------------------------------------------------------------------
####


def f3(supply_selected, df_distances_demand):
    """
    Calcula la diferencia entre el número máximo y mínimo de demandas asignadas a los puntos de suministro seleccionados,
    de forma más eficiente.
    """
    asignacion = df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
    counts = asignacion.value_counts()
    return counts.max() - counts.min()


def local_search_f3(solution, value, m, df_distances_demand, n_veces):
    """
    Realiza una búsqueda local para mejorar la solución minimizando la diferencia entre el número
    máximo y mínimo de demandas asignadas a un punto de suministro.
    """
    best_solution = solution[:]
    best_objective = value
    len_solution=len(solution)
    while True:
        for i in range(len_solution):
            for j in range(m):
                if j not in solution:
                    temp_solution = solution[:]
                    temp_solution[i] = j
                    temp_objective = f3(temp_solution, df_distances_demand)

                    if temp_objective < best_objective:
                        best_objective = temp_objective
                        best_solution = temp_solution[:]
                        improved=True
                        count+=1

            if count==n_veces:
                return best_solution, best_objective
            if improved:
                count_until_improve=0
            else:
                count_until_improve+=1
                if count_until_improve==len_solution:
                    return best_solution, best_objective



def add_solutions(solution, f1, f2, f3, route_solutions, df_solutions):
    solution = str(sorted(solution))

    if not df_solutions.empty:
        if solution in df_solutions["solution"].values:
            return  # La solución ya existe, no hacer nada

        df_dominado = df_solutions.copy()
        df_dominado = df_dominado[df_dominado["f1"] <= f1]
        df_dominado = df_dominado[df_dominado["f2"] <= f2]
        df_dominado = df_dominado[df_dominado["f3"] <= f3]
        if df_dominado.empty:
            # Eliminar soluciones que sean dominadas por la nueva
            df_solutions = df_solutions[
                ~(
                    (df_solutions["f1"] >= f1)
                    & (df_solutions["f2"] >= f2)
                    & (df_solutions["f3"] >= f3)
                    & (
                        (df_solutions["f1"] > f1)
                        | (df_solutions["f2"] > f2)
                        | (df_solutions["f3"] > f3)
                    )
                )
            ]
            # Agregar la nueva solución
            new_solution = pd.DataFrame(
                [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
            )
            print(new_solution)
            df_solutions = pd.concat([df_solutions, new_solution], ignore_index=True)
            df_solutions.to_csv(route_solutions, index=False)
            solucion_encontrada=True
    else:
        df_solutions = pd.DataFrame(
            [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
        )
        df_solutions.to_csv(route_solutions, index=False)
        solucion_encontrada=False
    return solucion_encontrada


####
# Multi Armed Bandit#------------------------------------------------------------------------------------------
####


def select_arm(context, betha, n_arms, weights):
    if np.random.rand() < betha:  # Exploración: acción aleatoria
        return np.random.randint(0, n_arms)
    else:  # Explotación: mejor acción según el modelo
        # Asegurarse de que el contexto sea un array numpy y tenga la forma correcta
        context_np = np.array(context).reshape(1, -1)  # Convertir a fila vector

        # Calcular la recompensa esperada para cada brazo
        # La recompensa esperada para cada brazo es el producto punto de su vector de pesos y el contexto.
        expected_rewards = np.dot(weights, context_np.T).flatten()

        print(f"Recompensas esperadas para cada brazo: {expected_rewards}")

        exp_rewards = np.exp(expected_rewards - np.max(expected_rewards))
        probabilities = exp_rewards / np.sum(exp_rewards)

        print(f"Probabilidad de coger cada brazo: {probabilities}")

        # Seleccionar un brazo aleatoriamente basado en estas probabilidades
        # np.random.choice permite elegir un elemento de una lista
        # con probabilidades especificadas.
        return np.random.choice(n_arms, p=probabilities)


def decode_action(chosen_arm, parameters=["f1", "f2", "f3"], k_values=[1, 2, 3, 4, 5]):
    param_idx = chosen_arm // len(k_values)
    k_idx = chosen_arm % len(k_values)
    return parameters[param_idx], k_values[k_idx]

def evaluate_solution(f1, f2, f3, route_solutions):

    return 1

def extraer_contexto(f1, f2, f3, route_solutions):

    return (10,10,10)


def update(chosen_arm, context, reward, weights, learning_rate):
    # Asegurarse de que el contexto sea un array numpy
    context_np = np.array(context)

    print(f"Pesos antes: {weights[chosen_arm]}")

    # Actualizar los pesos del brazo elegido.
    # Multiplicamos la recompensa directamente por el contexto y la tasa de aprendizaje.
    # Una recompensa positiva y un contexto dado harán que los pesos se ajusten
    # para favorecer ese brazo en contextos similares.
    # Una recompensa negativa (o baja) hará que los pesos se ajusten en la dirección opuesta,
    # desfavoreciendo ese brazo en contextos similares.
    weights[chosen_arm] += learning_rate * reward * context_np

    print(f"Pesos después: {weights[chosen_arm]}")

    return weights


####
# GRASP#------------------------------------------------------------------------------------------
####


def multi_GRASP(archive, k, m,context_size, max_iterations=5, alpha=1.0, betha=0.2, i=0):

    n_arms=context_size*max_iterations

    folder_distances = "./data/distances/demand/"
    route_distances = folder_distances + archive + ".csv"
    df_distances_demand = pd.read_csv(route_distances)

    folder_solutions = f"Solutions/Multiprocessing/{archive}/"
    # Crea la carpeta si no existe
    os.makedirs(folder_solutions, exist_ok=True)

    route_solutions = folder_solutions + archive + f"_#{i}" + ".csv"

    if os.path.exists(route_solutions):
        df_solutions = pd.read_csv(route_solutions)
        print(df_solutions)
    else:
        columnas = ["solution", "f1", "f2", "f3"]
        df_solutions = pd.DataFrame(columns=columnas)
        print(df_solutions)

    # Inicializo los pesos para el MAB

    w__dir=f'Weights/{archive}'
    os.makedirs(w__dir, exist_ok=True)
    route_weights = w__dir + archive + f"_#{i}" + ".npy"
    if os.path.exists(route_weights):
        weights = np.load(route_weights)
        print(f'había pesos: {weights}')
    else:
        weights = np.zeros((n_arms, context_size))
        print(f'No había pesos')



    greedy_algorithm = Greedy(df_distances_demand, k, m, alpha)
    """
    Algoritmo GRASP con dos búsquedas locales elegidas aleatoriamente (sin repetición).
    Se mide y muestra el tiempo de ejecución de cada búsqueda local.
    """
    # Etapa Greedy inicial
    solution, f1_value = greedy_algorithm.run()
    f2_value = f2(solution, df_distances_demand)
    f3_value = f3(solution, df_distances_demand)

    solucion_encontrada=add_solutions(solution, f1_value, f2_value, f3_value, route_solutions, df_solutions)

    if solucion_encontrada:
        return solution
    else:
        value_prev=evaluate_solution(f1_value, f2_value, f3_value)

    # El contexto es la mejora que hay que hacer a f1 para ser solucion, a f2 para ser solucion, para f3 para ser solucion

    context = extraer_contexto(f1, f2, f3, route_solutions)

    chosen_arm=select_arm(context, betha, n_arms, weights)
    funcion, n_veces = decode_action(chosen_arm, parameters=["f1", "f2", "f3"], k_values=[1, 2, 3, 4, 5])

    # Ejecutar primera búsqueda local
    if funcion == "f1":
        solution, f1_value = local_search_f1(solution, f1_value, m, df_distances_demand, n_veces)
        f2_value = f2(solution, df_distances_demand)
        f3_value = f3(solution, df_distances_demand)
    elif funcion == "f2":
        solution, f2_value = local_search_f2(solution, f2_value, m, df_distances_demand, n_veces)
        f1_value = f1(solution, df_distances_demand)
        f3_value = f3(solution, df_distances_demand)
    elif funcion == "f3":
        solution, f3_value = local_search_f3(solution, f3_value, m, df_distances_demand, n_veces)
        f1_value = f1(solution, df_distances_demand)
        f2_value = f2(solution, df_distances_demand)

    solucion_encontrada=add_solutions(solution, f1_value, f2_value, f3_value, route_solutions, df_solutions)

    if solucion_encontrada:
        reward=5
    else:
        value_next=evaluate_solution(f1_value, f2_value, f3_value)
        reward=(value_next-value_prev)/3
    
    update(chosen_arm, context,reward)

    return solution


In [ ]:
context_size=3
max_iterations=5

In [ ]:
MAB = MultiArmed_Bandit(n_arms, context_size)

In [ ]:
context=(20,100,50)

In [ ]:
chosen_arm= MAB.select_arm(context)

Recompensas esperadas para cada brazo: [0.  0.  0.  0.  0.  0.  0.  0.  4.3 0.  0.  0.  0.  0.  0. ]
Probabilidad de coger cada brazo: [0.01140254 0.01140254 0.01140254 0.01140254 0.01140254 0.01140254
 0.01140254 0.01140254 0.8403645  0.01140254 0.01140254 0.01140254
 0.01140254 0.01140254 0.01140254]


In [ ]:
funcion, n_veces = MAB.decode_action(chosen_arm)

In [ ]:
print(f'Ahora mejoramos la funcion {funcion} {n_veces} veces, pongamos de ejemplo que el resultado es 400, 230, 27.')

Ahora mejoramos la funcion f2 4 veces, pongamos de ejemplo que el resultado es 400, 230, 27.


In [ ]:
valor_sol_prev=2.5
valor_sol_actual=2.6

reward=(valor_sol_actual-valor_sol_prev)/3

In [ ]:
MAB.update(chosen_arm, context,reward)

Pesos antes: [0.00666667 0.03333333 0.01666667]
Pesos después: [0.01333333 0.06666667 0.03333333]
